# Combine the CSVs into 1

In [ ]:
# imports
import pandas as pd
import numpy as np
import os

Get the average of the duplicates, and parse them by column followed by row

Dot swapping to bring the dots back to their original position (8*18, max size 144)
* For panel 1:
    * K15/16 to C1/2 (BB w/o urea) - 136 to 2
    * M15/16 to K1/2 (IgE 62.5) - 138 to 10
    * O15/16 to I5/6 (Chicken breast) - 140 to 44
    * L15/16 to D1/2 (PBS) - 137 to 3
    * N15/16 to L1/2 (IgE 31.25) - 139 to 11
    * P15/16 to J5/6 (Cooked soy) - 141 to 45
    * Delete last 8 elements (136 onwards)

* For panel 2: (a bit more complicated)
    * Swap I3/4 with K3/4 (BSF PKM WT with BB w urea gel) (28 with 26)
    * Swap H3/4 with L3/4 (BSF M<P WT with BB w/o urea gel) (25 with 29)
    * Just delete the empty dots
    * A5/6, E5/6, G5/6, I5/6, K5/6 (36, 40, 42, 44, 46)
    * B5/6, F5/6, H5/6, J5/6, L5/6 (37, 41, 43, 45, 47)
    * Rearrange as I see fit later on

In [ ]:
# Set directory of batch
batch_directory = "150222-CSV"
# List of dataframes of values
batch1 = []
batch2 = []
# Record panel
panels = []
# Record ID
id1 = []
id2 = []

print(f"Number of membranes: {len(os.listdir(batch_directory))}")

for file in os.listdir(batch_directory):
    # Get panel number and id
    panel, idx = file.split("-")
    idx = idx[:len(idx)-4]
    panels.append(panel)

    # Use only the first 2 columns with usecols
    csv = pd.read_csv(batch_directory + '/' + file, 
                      usecols=[0,1], 
                      names=['Spot', 'Value']) # Must be the full file path
    # Remove top 9 rows as they are irrelevant
    csv = csv.dropna().iloc[4:,].reset_index(drop=True)
    # Split Spot into Row and Column
    csv['Row'] = csv['Spot'].str[4]
    csv['Col'] = csv['Spot'].str[5:]
    # Handle value error if values are in scientific notation
    try:
        csv['Value'].astype(int)
    except ValueError:
        print(f'Notation error: File {file} needs to be converted to int!')
        continue

    # Convert them into list of values
    # We process them column by column, and average the duplicates
    l = []
    for col in range(1, 9): # We use 2*i - 1 and 2*i
        for row in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R']:
            spot1 = "Spot" + row + str(2*col - 1)
            spot2 = "Spot" + row + str(2*col)
            dot1 = int(csv.loc[csv['Spot'] == spot1, 'Value'].iloc[0])
            dot2 = int(csv.loc[csv['Spot'] == spot2, 'Value'].iloc[0])
            dot = (dot1 + dot2) / 2
            l.append(dot)

    # Swap the dots back to their original position
    # Swap panel 1
    if panel == "1":
        l[2] = l[136]
        # l[10] = l[138]
        l[44] = l[140]
        l[3] = l[137]
        # l[11] = l[139]
        l[45] = l[141]
        del l[136:]

        batch1.append(l)
        id1.append(idx)
        
    # Delete panel 2 empty
    if panel == "2":
        print(l[28], l[26], l[25], l[29])
        # Swap
        l[28], l[26] = l[26], l[28]
        l[25], l[29] = l[29], l[25]
        print(l[28], l[26], l[25], l[29])
        # Delete 36 and 37
        del l[36:38]
        # Delete 40 to 49
        del l[38:46]

        batch2.append(l)
        id2.append(idx)

Number of membranes: 60
178576.5 169297.0 163987.5 177806.0
169297.0 178576.5 177806.0 163987.5
165607.5 154597.0 147975.5 167754.5
154597.0 165607.5 167754.5 147975.5
513799.5 522256.5 516444.0 510863.0
522256.5 513799.5 510863.0 516444.0
479377.0 486847.0 482201.5 479116.0
486847.0 479377.0 479116.0 482201.5
1137580.0 1114145.0 1099810.0 1141140.0
1114145.0 1137580.0 1141140.0 1099810.0
203215.0 200855.5 189413.5 202135.0
200855.5 203215.0 202135.0 189413.5
526290.0 516852.0 512376.0 525881.0
516852.0 526290.0 525881.0 512376.0
232823.0 225338.5 216961.5 232899.5
225338.5 232823.0 232899.5 216961.5
434765.5 446986.5 434717.0 413927.0
446986.5 434765.5 413927.0 434717.0
438040.5 434820.0 426076.0 437184.5
434820.0 438040.5 437184.5 426076.0
171787.0 179006.0 172562.5 166314.0
179006.0 171787.0 166314.0 172562.5
698723.5 695508.5 687399.5 698886.0
695508.5 698723.5 698886.0 687399.5
552512.5 546807.0 544116.0 551094.5
546807.0 552512.5 551094.5 544116.0
172113.5 170362.5 171962.5 17211

In [ ]:
print(len(batch2[0]))

134


## Save each panel's CSV separately, and do the normalisation separately

In [ ]:
# For panel 1
panel1 = pd.DataFrame.from_records(batch1)

# Add batch date
panel1.insert(0, "Batch", batch_directory)
# Add panel number
panel1.insert(0, "Panel", 1)
# Add ID
panel1.insert(0, "ID", id1)

panel1

,ID,Panel,Batch,0,1,2,3,4,5,6,...,126,127,128,129,130,131,132,133,134,135
0,30,1,150222-CSV,275929.5,273712.0,278562.0,277234.5,261851.0,262740.0,350092.0,...,272664.0,270047.0,264732.5,264959.0,265451.5,266696.5,266357.0,267911.5,272430.5,273096.0
1,5,1,150222-CSV,207067.0,203687.0,202864.5,202691.0,184140.0,181549.0,255618.0,...,215587.0,208414.5,204950.5,203018.5,204633.5,202623.0,202989.5,202083.0,203006.5,201396.0
2,2,1,150222-CSV,316393.0,245686.5,192628.0,194511.0,173604.5,174264.0,232959.5,...,203795.5,185624.0,187149.5,184896.5,181384.5,184017.0,184819.0,184738.0,196037.0,189384.0
3,168,1,150222-CSV,216432.5,203909.5,196577.5,198888.5,193620.5,189475.0,293384.5,...,215597.5,200138.0,195792.5,191446.5,188850.0,185772.5,185657.5,187802.5,199538.5,193515.5
4,61,1,150222-CSV,332340.5,326788.5,329644.5,330614.5,313646.0,314211.5,414414.5,...,325486.5,316998.0,315434.5,312708.5,316768.0,316668.5,320041.5,320643.5,325363.0,325841.0
5,14,1,150222-CSV,360785.5,362059.0,372381.0,372864.0,347659.5,342489.5,402313.5,...,380295.5,374138.5,372376.5,368809.5,369974.5,367968.5,368094.0,367214.0,370032.5,370655.5
6,220,1,150222-CSV,279525.0,240902.0,207959.5,209578.5,189970.5,191756.5,285277.5,...,219489.5,209542.0,210231.0,205590.0,207105.5,207704.0,204323.0,203712.5,204766.5,205544.0
7,142,1,150222-CSV,708788.5,703801.5,713389.0,714972.0,690790.0,691195.5,805841.5,...,710237.0,699282.0,699966.0,697242.0,699591.5,698411.0,700535.0,700045.0,706924.0,708434.0
8,169,1,150222-CSV,737439.5,718947.0,718155.5,721511.0,703168.0,702354.5,790045.0,...,715103.5,705472.5,704018.0,702209.5,702567.0,703123.5,705571.5,709199.0,713010.5,714883.5
9,127,1,150222-CSV,251338.5,232795.0,245453.0,249671.0,230574.0,228173.0,333984.0,...,246606.0,237656.5,238686.0,237163.0,239732.0,238637.0,239818.0,240360.5,243488.0,242887.5


In [ ]:
# For panel 2
panel2 = pd.DataFrame.from_records(batch2)
# Add batch date
panel2.insert(0, "Batch", batch_directory)
# Add panel number
panel2.insert(0, "Panel", 2)
# Add ID
panel2.insert(0, "ID", id2)

panel2

,ID,Panel,Batch,0,1,2,3,4,5,6,...,124,125,126,127,128,129,130,131,132,133
0,74,2,150222-CSV,245137.5,197786.5,183180.0,181943.0,182466.5,177781.5,263686.0,...,199806.0,201433.5,207541.0,203371.0,206317.0,215250.5,246748.5,217893.5,232336.5,231992.5
1,B,2,150222-CSV,176295.0,172086.5,167327.0,160230.5,159983.5,157171.5,237221.0,...,194098.0,196349.5,194478.0,197290.0,199247.0,200434.5,203493.0,210738.0,216945.0,228041.5
2,158,2,150222-CSV,549201.0,530645.0,531764.0,527475.5,525357.0,523609.5,625937.5,...,536183.0,543685.0,555127.0,542944.5,544272.0,566051.5,578027.5,557047.5,568871.0,560612.0
3,234,2,150222-CSV,575688.0,512179.0,496709.0,493973.0,494356.5,494340.0,591651.5,...,502632.0,513047.5,521347.5,509330.5,513986.5,524146.0,549498.0,520841.5,534906.5,527526.0
4,4,2,150222-CSV,1177495.0,1170845.0,1155535.0,1147420.0,1143915.0,1134840.0,1334620.0,...,1199235.0,1201405.0,1215470.0,1209295.0,1218680.0,1231015.0,1306175.0,1231940.0,1253345.0,1260155.0
5,10,2,150222-CSV,205711.0,204848.0,198060.5,193825.0,191453.5,189629.0,270877.5,...,219672.0,219351.0,224167.0,228741.0,234824.0,243391.0,264374.0,239662.0,256755.5,254138.0
6,219,2,150222-CSV,547005.5,522197.0,526937.5,526158.0,525011.0,521823.5,600406.0,...,543039.5,556574.0,561528.5,549659.5,552591.5,570972.5,596442.0,559320.5,571094.5,570186.0
7,217,2,150222-CSV,330394.5,243980.0,236609.0,234619.0,234264.0,229225.5,323902.0,...,250304.5,254870.5,261612.5,258666.5,261678.0,274104.0,313843.5,267636.5,275676.5,279815.5
8,122,2,150222-CSV,630828.5,577124.5,528093.0,507641.0,508318.0,494884.5,762126.5,...,478463.0,460652.5,457798.0,457663.0,476070.0,532911.0,583731.0,544256.0,576694.5,601491.5
9,235,2,150222-CSV,465266.5,452600.0,446668.0,446341.0,446948.5,443633.0,536409.5,...,462359.0,474599.0,492166.0,467807.0,469900.0,479365.5,507007.0,472498.5,493663.5,482848.0


## Save the combined file to First Stage Combine folder

In [ ]:
# Make new directory if it doesnt exist
from pathlib import Path
Path("First Stage Combine/" + batch_directory).mkdir(parents=True, exist_ok=True)

# Save panels
panel1.to_csv("First Stage Combine/" + batch_directory + "/1-combine.csv", index=False)
panel2.to_csv("First Stage Combine/" + batch_directory + "/2-combine.csv", index=False)

Now we do the normalisation in R

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=72487218-bfd4-4a63-a450-c076490888ed' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>